In [1]:
import os
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
import numpy as np

In [2]:
root = 'E:/wx/2019上课题/数据/exp'
file_list = os.listdir(root)

In [11]:
# 重写表达谱文件
def rewrite_exp():
    genelist = pd.read_csv('data/symbol_nodes.csv', sep='\t', header=None)
    hgncFile = pd.read_csv('E:/wx/2019上课题/数据/hgnc_complete_set.txt', sep='\t',
                               usecols=['symbol', 'ensembl_gene_id', ],
                               engine='python')
    symbol2ensg = hgncFile.loc[hgncFile['symbol'].isin(genelist[0])]
    ensembl_list = list(symbol2ensg['ensembl_gene_id'])

    all_exp = pd.DataFrame()
    for file in file_list:
        exp_list = []
        # 合并文件
        print(file)
        with open(os.path.join(root,file), 'r') as file:
            headerline = file.readline().strip().split('\t')
            for f in file.readlines():
                enseID = f.strip().split('\t')
                ensgid = enseID[0].split('.')[0]
                if ensgid in ensembl_list:
                    symbolname = symbol2ensg.loc[symbol2ensg['ensembl_gene_id']==ensgid, 'symbol'].values[0]
                    tmp = enseID[1:]
                    tmp.insert(0, str(symbolname))
                    exp_list.append(tmp)
        exp_mat = pd.DataFrame(exp_list, columns=headerline)
        exp_mat = exp_mat.set_index(exp_mat['Ensembl_ID']).drop(['Ensembl_ID'], axis = 1).astype('float')
        all_exp = pd.concat([all_exp, exp_mat], axis=1)
    all_exp.to_csv('data/exp_data.csv')

In [12]:
rewrite_exp()

TCGA-ACC.htseq_fpkm-uq.tsv
TCGA-BLCA.htseq_fpkm-uq.tsv
TCGA-BRCA.htseq_fpkm-uq.tsv
TCGA-CESC.htseq_fpkm-uq.tsv
TCGA-CHOL.htseq_fpkm-uq.tsv
TCGA-COAD.htseq_fpkm-uq.tsv
TCGA-DLBC.htseq_fpkm-uq.tsv
TCGA-ESCA.htseq_fpkm-uq.tsv
TCGA-GBM.htseq_fpkm-uq.tsv
TCGA-HNSC.htseq_fpkm-uq.tsv
TCGA-KICH.htseq_fpkm-uq.tsv
TCGA-KIRC.htseq_fpkm-uq.tsv
TCGA-KIRP.htseq_fpkm-uq.tsv
TCGA-LGG.htseq_fpkm-uq.tsv
TCGA-LIHC.htseq_fpkm-uq.tsv
TCGA-LUAD.htseq_fpkm-uq.tsv
TCGA-LUSC.htseq_fpkm-uq.tsv
TCGA-MESO.htseq_fpkm-uq.tsv
TCGA-OV.htseq_fpkm-uq.tsv
TCGA-PAAD.htseq_fpkm-uq.tsv
TCGA-PCPG.htseq_fpkm-uq.tsv
TCGA-PRAD.htseq_fpkm-uq.tsv
TCGA-READ.htseq_fpkm-uq.tsv
TCGA-SARC.htseq_fpkm-uq.tsv
TCGA-SKCM.htseq_fpkm-uq.tsv
TCGA-STAD.htseq_fpkm-uq.tsv
TCGA-TGCT.htseq_fpkm-uq.tsv
TCGA-THCA.htseq_fpkm-uq.tsv
TCGA-THYM.htseq_fpkm-uq.tsv
TCGA-UCEC.htseq_fpkm-uq.tsv
TCGA-UCS.htseq_fpkm-uq.tsv
TCGA-UVM.htseq_fpkm-uq.tsv


In [4]:
file_labels = dict(zip(file_list, list(range(1,len(file_list)+1))))
samples_list = []

for file in file_list:
    with open(os.path.join(root,file), 'r') as f:
        first_row = f.readline()
        first_row = first_row.strip().split('\t')
        for sample in first_row[1:]:
            if int(sample.strip()[-3:-2]) == 0:
                # case
                samples_list.append((sample, file_labels[file]))
            else:
                # control
                samples_list.append((sample, 0))

In [5]:
sample2labels = pd.DataFrame(samples_list,columns=['sampleID', 'label'])
# sample2labels.loc[sample2labels['label']==0]
# pd.value_counts(sample2labels['label'])

In [10]:
sample2labels.shape

(10906, 2)

In [9]:
'TCGA-G5-6641-01A' in list(sample2labels['sampleID'])

True

In [6]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(np.array(sample2labels['sampleID']).reshape(-1, 1), 
                                          np.array(sample2labels['label'])
                                         ) # 过采样
# from collections import  Counter
# Counter(y_resampled)

In [7]:
allsamples = pd.DataFrame({'sampleID':X_resampled[:,0], 'label':y_resampled})

In [8]:
length = allsamples.shape[0]
index_list = list(range(length))

In [9]:
import random
train_num, validation_num, test_num= 0.7, 0.2, 0.1
random.shuffle(index_list)
train_index = index_list[0:int(length*train_num)]
validation_index = index_list[int(length*train_num): int(length*train_num) + int(length*validation_num)]
test_index = index_list[int(length * train_num) + int(length * validation_num):]

train_samples = allsamples.loc[train_index]
validation_samples = allsamples.loc[validation_index]
test_samples = allsamples.loc[test_index]

In [12]:
train_samples.head()

,sampleID,label
30307,TCGA-BF-A3DJ-01A,25
13577,TCGA-VS-A9UP-01A,4
35813,TCGA-V4-A9EM-01A,32
21660,TCGA-E1-A7Z3-01A,14
23861,TCGA-66-2770-01A,17


In [10]:
e = pd.read_csv('data/exp_data.csv')


In [79]:
e[train_samples.head()['sampleID']]

,TCGA-BF-A3DJ-01A,TCGA-VS-A9UP-01A,TCGA-V4-A9EM-01A,TCGA-E1-A7Z3-01A,TCGA-66-2770-01A
0,10.056254,10.802741,6.574110,8.048858,12.647654
1,20.799541,21.219319,23.871829,20.807075,21.236522
2,16.787765,16.833507,17.250954,16.641846,16.913446
3,14.927467,11.450758,12.853385,16.061449,14.049444
4,0.000000,0.000000,0.000000,0.000000,9.136180
5,19.079686,21.619664,19.043656,21.269202,21.491262
6,20.138343,18.775063,21.341693,19.567137,20.467054
7,8.352366,0.000000,0.000000,0.000000,0.000000
8,21.613759,19.710384,20.708761,18.386985,18.758029
9,16.122463,14.907417,13.452357,15.893695,17.743953


In [71]:
def data_iter_exp(batch_size, dataset, exp):
    dataset = dataset.reset_index(drop=True)
    sam_l = list(dataset['sampleID'])
    lab_l = list(dataset['label'])
    num_samples= dataset.shape[0]
    indices = list(range(num_samples))
    random.shuffle(indices)
    for i in range(0, num_samples, batch_size):
        j = indices[i: min(i + batch_size, num_samples)]
        res = dataset.loc[j]
        x = exp[res['sampleID']]
        y = res['label']
        yield nd.array(x).transpose().reshape(len(j),x.shape[0],1), nd.array(res['label'])

In [73]:
for x,y in data_iter(1,train_samples, e):
    print(x, y)
    break


[[[ 8.488233]
  [21.469767]
  [16.52824 ]
  ...
  [ 0.      ]
  [15.417712]
  [ 0.      ]]]
<NDArray 1x7394x1 @cpu(0)> 
[18.]
<NDArray 1 @cpu(0)>



[22. 23.  8.]
<NDArray 3 @cpu(0)>

In [57]:
x

,TCGA-HC-7230-01A,TCGA-DY-A1DE-01A,TCGA-LN-A49O-01A
0,7.327228,10.976172,11.817954
1,22.274467,22.146188,21.902150
2,17.979646,17.438881,17.445868
3,17.326319,14.593624,14.470374
4,13.799346,16.590440,0.000000
5,19.387006,19.168368,20.411045
6,20.025025,19.956621,20.434961
7,0.000000,9.846859,0.000000
8,18.049484,18.720317,18.560671
9,14.936393,18.299574,18.787224


In [36]:
dataset.loc[dataset['sampleID'].isin(j)].drop_duplicates()

,sampleID,label
36144,TCGA-VD-A8KE-01A,32
2484,TCGA-D5-6537-01A,6
13603,TCGA-Q1-A5R1-01A,4


In [49]:
sorted(dataset.index)

[1,
 3,
 4,
 6,
 7,
 10,
 11,
 12,
 13,
 14,
 16,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28,
 29,
 30,
 31,
 32,
 34,
 37,
 38,
 39,
 40,
 41,
 45,
 46,
 47,
 48,
 51,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 71,
 72,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 82,
 83,
 84,
 85,
 86,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 108,
 109,
 110,
 111,
 113,
 115,
 117,
 119,
 121,
 122,
 124,
 125,
 126,
 128,
 129,
 130,
 131,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 144,
 146,
 147,
 149,
 150,
 152,
 154,
 156,
 157,
 158,
 159,
 161,
 162,
 165,
 166,
 170,
 171,
 173,
 174,
 175,
 176,
 178,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 192,
 193,
 194,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 209,
 211,
 212,
 213,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 226,
 227,
 228,
 229,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 